In [2]:
import numpy as np
import pandas as pd
import altair as alt

from tqdm import tqdm

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
from clickhouse_driver import Client as Clickhouse

def click_query(q, **kw):
    click = Clickhouse("localhost")
    return click.query_dataframe(q, params=kw)

In [4]:
click_query("DESCRIBE TABLE obs_web")

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,measurement_uid,String,,,,,
1,input,Nullable(String),,,,,
2,report_id,String,,,,,
3,measurement_start_time,DateTime64(6),,,,,
4,software_name,String,,,,,
5,software_version,String,,,,,
6,test_name,String,,,,,
7,test_version,String,,,,,
8,probe_asn,Int32,,,,,
9,probe_cc,String,,,,,


In [5]:
click_query("DESCRIBE TABLE obs_web_ctrl")

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,measurement_uid,String,,,,,
1,input,Nullable(String),,,,,
2,report_id,String,,,,,
3,measurement_start_time,DateTime64(6),,,,,
4,software_name,String,,,,,
5,software_version,String,,,,,
6,test_name,String,,,,,
7,test_version,String,,,,,
8,hostname,String,,,,,
9,observation_id,String,,,,,


In [28]:
MEASUREMENT_START_DAY = '2024-04-01'
MEASUREMENT_END_DAY = '2024-07-01'
ANALYSIS_COUNTRY_CODES = [
    "MM"
]

In [6]:
%%time
df = click_query("""
SELECT 
report_id,
input,
measurement_uid,
probe_cc,
probe_asn,
probe_as_org_name,
probe_as_cc,
network_type,
measurement_start_time,
hostname,
test_name,
ip,
port,
ip_asn,
ip_as_org_name,
resolver_ip,
resolver_cc,
resolver_asn,
resolver_as_org_name,
resolver_as_cc,
dns_engine,
dns_failure,
dns_answer,
dns_answer_type,
http_request_url,
http_failure,
http_runtime,
http_response_body_sha1,
http_response_status_code,
http_response_body_length,
http_response_body_is_truncated,
probe_analysis
FROM obs_web
WHERE probe_cc = 'MM' 
AND measurement_start_time > '2024-04-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'web_connectivity'
""")

CPU times: user 7.03 s, sys: 513 ms, total: 7.54 s
Wall time: 1min 25s


In [7]:
pd.set_option('display.max_colwidth', None)
df['explorer_url'] = "https://explorer.ooni.org/measurement/" + df['measurement_uid']

In [8]:
df.to_csv('2024-MM-df.csv')

In [5]:
df = pd.read_csv('2024-MM-df.csv')

In [9]:
df[['probe_asn', 'probe_as_org_name']].drop_duplicates(subset=['probe_asn']).to_csv('2024_MM_ASN_Lookup.csv')

In [10]:
len(df)

128871

In [11]:
df

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,dns_answer_type,http_request_url,http_failure,http_runtime,http_response_body_sha1,http_response_status_code,http_response_body_length,http_response_body_is_truncated,probe_analysis,explorer_url
0,20240401T003750Z_webconnectivity_MM_136255_n1_4F0EJvo3L8aJzlPF,https://twitter.com/,20240401003850.725383_MM_webconnectivity_59c5e512edc11b74,MM,136255,Telecom International Myanmar Company Limited,MM,wifi,2024-04-01 00:38:49,twitter.com,web_connectivity,104.244.42.193,443.0,13414.0,Twitter Inc.,74.125.190.131,SG,15169,Google LLC,US,system,None,104.244.42.193,A,None,None,None,None,NaN,NaN,NaN,tcp_ip,https://explorer.ooni.org/measurement/20240401003850.725383_MM_webconnectivity_59c5e512edc11b74
1,20240401T003750Z_webconnectivity_MM_136255_n1_4F0EJvo3L8aJzlPF,https://twitter.com/,20240401003850.725383_MM_webconnectivity_59c5e512edc11b74,MM,136255,Telecom International Myanmar Company Limited,MM,wifi,2024-04-01 00:38:49,twitter.com,web_connectivity,104.244.42.1,443.0,13414.0,Twitter Inc.,74.125.190.131,SG,15169,Google LLC,US,system,None,104.244.42.1,A,None,None,None,None,NaN,NaN,NaN,tcp_ip,https://explorer.ooni.org/measurement/20240401003850.725383_MM_webconnectivity_59c5e512edc11b74
2,20240401T003750Z_webconnectivity_MM_136255_n1_4F0EJvo3L8aJzlPF,https://twitter.com/,20240401003850.725383_MM_webconnectivity_59c5e512edc11b74,MM,136255,Telecom International Myanmar Company Limited,MM,wifi,2024-04-01 00:38:49,twitter.com,web_connectivity,104.244.42.65,443.0,13414.0,Twitter Inc.,74.125.190.131,SG,15169,Google LLC,US,system,None,104.244.42.65,A,None,None,None,None,NaN,NaN,NaN,tcp_ip,https://explorer.ooni.org/measurement/20240401003850.725383_MM_webconnectivity_59c5e512edc11b74
3,20240401T003750Z_webconnectivity_MM_136255_n1_4F0EJvo3L8aJzlPF,https://twitter.com/,20240401003850.725383_MM_webconnectivity_59c5e512edc11b74,MM,136255,Telecom International Myanmar Company Limited,MM,wifi,2024-04-01 00:38:49,twitter.com,web_connectivity,104.244.42.129,443.0,13414.0,Twitter Inc.,74.125.190.131,SG,15169,Google LLC,US,system,None,104.244.42.129,A,None,None,None,None,NaN,NaN,NaN,tcp_ip,https://explorer.ooni.org/measurement/20240401003850.725383_MM_webconnectivity_59c5e512edc11b74
4,20240401T003750Z_webconnectivity_MM_136255_n1_4F0EJvo3L8aJzlPF,https://twitter.com/,20240401003850.725383_MM_webconnectivity_59c5e512edc11b74,MM,136255,Telecom International Myanmar Company Limited,MM,wifi,2024-04-01 00:38:49,twitter.com,web_connectivity,None,NaN,NaN,None,74.125.190.131,SG,15169,Google LLC,US,None,None,None,None,https://twitter.com/,generic_timeout_error,None,None,0.0,NaN,0.0,tcp_ip,https://explorer.ooni.org/measurement/20240401003850.725383_MM_webconnectivity_59c5e512edc11b74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128866,20240425T085511Z_webconnectivity_MM_9988_n1_wdXrgbqW4v6WxXI5,https://radiofreel.gitlab.io/,20240425090300.887848_MM_webconnectivity_2eceae7aab37b2a4,MM,9988,Myanma Post & Telecommunication,MM,unknown,2024-04-25 09:03:03,radiofreel.gitlab.io,web_connectivity,35.185.44.232,443.0,396982.0,Google LLC,172.253.236.195,SG,15169,Google LLC,US,system,None,35.185.44.232,A,None,None,None,None,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240425090300.887848_MM_webconnectivity_2eceae7aab37b2a4
128867,20240425T085511Z_webconnectivity_MM_9988_n1_wdXrgbqW4v6WxXI5,https://radiofreel.gitlab.io/,20240425090300.887848_MM_webconnectivity_2eceae7aab37b2a4,MM,9988,Myanma Post & Telecommunication,MM,unknown,2024-04-25 09:03:03,radiofreel.gitlab.io,web_connectivity,None,NaN,NaN,None,172.253.236.195,SG,15169,Google LLC,US,None,None,None,None,https://radiofreel.gitlab.io/,None,None,0a43e591

In [12]:
df['probe_analysis'].value_counts()

false           110871
tcp_ip            7433
dns               5337
http-failure      1303
http-diff          416
Name: probe_analysis, dtype: int64

In [13]:
df_dns_answer = df[df['dns_answer_type']=='A'].drop_duplicates(subset=['measurement_uid'])

In [14]:
len(df_dns_answer)

31358

In [15]:
dns_resp_sorted = df_dns_answer[
    (df_dns_answer['probe_analysis'] == 'dns')
].groupby('dns_answer')['hostname'].nunique().sort_values().reset_index()

In [16]:
dns_resp_sorted[
    dns_resp_sorted['hostname'] > 2
]

,dns_answer,hostname
29,59.153.90.11,3
30,0.0.0.0,12
31,103.105.174.6,38
32,127.0.0.1,255


In [17]:
dns_resp_sorted[
    dns_resp_sorted['hostname'] > 10
]

,dns_answer,hostname
30,0.0.0.0,12
31,103.105.174.6,38
32,127.0.0.1,255


In [18]:
dns_suspicious = dns_resp_sorted[dns_resp_sorted['hostname'] > 10]['dns_answer'].values.tolist()
dns_suspicious


['0.0.0.0', '103.105.174.6', '127.0.0.1']

In [19]:
df_dns_answer[df_dns_answer['dns_answer'].isin(dns_suspicious)].drop_duplicates(subset=['dns_answer']).to_csv('2024_MM_dns_suspicious.csv')

In [20]:
dns_answer_filter = df_dns_answer[df_dns_answer['dns_answer'].isin(dns_suspicious)]

dns_answer_filter = dns_answer_filter.pivot_table(index=['dns_answer', 'probe_cc'], 
              values=['hostname'],
              aggfunc={'hostname': ['nunique', lambda x: ', '.join(set(x))]}).rename(columns = {
                                                                    '<lambda_0>': 'hostnames',
                                                                    'len' : 'hostname_count'}).reset_index()

dns_answer_filter.head()

dns_answer probe_cc  \
                            
0        0.0.0.0       MM   
1  103.105.174.6       MM   
2      127.0.0.1       MM   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [21]:
dns_answer_filter.to_csv('2024_MM_dns_answer_hostname')

In [22]:
df_http_answer = df[df['http_request_url'].notnull()].drop_duplicates(subset=['measurement_uid'])

In [23]:
http_domain_count = df_http_answer[
    df_http_answer['probe_analysis'] == 'http-diff'
].groupby('http_response_body_sha1')['hostname'].nunique().sort_values().reset_index()

In [24]:
http_domain_count[
    http_domain_count['hostname'] > 2
]

,http_response_body_sha1,hostname


In [25]:
http_domain_count[
    http_domain_count['hostname'] > 10
]

,http_response_body_sha1,hostname


In [26]:
http_suspicious = http_domain_count[http_domain_count['hostname'] > 2]['http_response_body_sha1'].values.tolist()
http_suspicious


[]

In [27]:
df_http_answer[df_http_answer['http_response_body_sha1'].isin(http_suspicious)].drop_duplicates(subset=['http_response_body_sha1']).to_csv('2024_MM_http_suspicious.csv')

In [29]:
confirmed_ips = [
    '167.172.4.60','103.105.174.6','176.103.130.135'
]

likely_ips = [
    '']

false_positive_ips = [
    ''
]

confirmed_http_resp = [
    ''
]

likely_http_resp = [
    '3e3ccd027a49db4b4efbe63054155033788239b5'
]

false_positive_http_resp = [
    'eb6ce822e8e9989755037c6838f3b100fd6815c3','0004d4f82b546013424b2e0de084395071eef98b','c6acd6b7401c703413c1d50799806a973ea19837'
]


In [31]:
%%time
df_with_failure = click_query("""
WITH multiIf(
    dns_failure IS NOT NULL, tuple('dns', dns_failure),
    tcp_failure IS NOT NULL, tuple('tcp', tcp_failure),
    tls_failure IS NOT NULL, tuple('tls', tls_failure),
    http_failure IS NOT NULL, tuple('https', http_failure),
    tuple('ok', '')
) as failure
SELECT 
report_id,
input,
measurement_uid,
probe_cc,
probe_asn,
probe_as_org_name,
probe_as_cc,
network_type,
measurement_start_time,
hostname,
test_name,
ip,
port,
ip_asn,
ip_as_org_name,
resolver_ip,
resolver_cc,
resolver_asn,
resolver_as_org_name,
resolver_as_cc,
dns_engine,
dns_failure,
dns_answer,
tcp_success,
tcp_failure,
tls_handshake_time,
tls_handshake_read_count,
tls_handshake_write_count,
tls_handshake_read_bytes,
tls_handshake_write_bytes,
tls_handshake_last_operation,
tls_cipher_suite IS NOT NULL as tls_success,
tls_is_certificate_valid,
tls_end_entity_certificate_subject,
tls_end_entity_certificate_subject_common_name,
tls_end_entity_certificate_issuer,
tls_end_entity_certificate_issuer_common_name,
tls_end_entity_certificate_san_list,
tls_end_entity_certificate_not_valid_after,
tls_end_entity_certificate_not_valid_before,
tls_failure,
http_response_body_sha1,
http_request_url,
http_failure,
failure.1 as failure_class,
IF(failure_class = 'ok', 'ok', concat(failure_class, '.', failure_str)) as failure_str_full,
IF(startsWith(failure.2, 'unknown_failure'), 'unknown_failure', failure.2) as failure_str,
failure.2 as failure_str_raw
FROM obs_web
WHERE probe_cc = 'MM' 
AND measurement_start_time > '2024-04-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'web_connectivity'
""")

CPU times: user 13.2 s, sys: 910 ms, total: 14.1 s
Wall time: 1min 29s


In [32]:
pd.set_option('display.max_colwidth', None)
df_with_failure['explorer_url'] = "https://explorer.ooni.org/m/" + df_with_failure['measurement_uid']

In [33]:
df_with_failure["probe_asn"] = "AS" + df_with_failure["probe_asn"].apply(str)

In [34]:
df_with_failure

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_handshake_time,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_success,tls_is_certificate_valid,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_failure,http_response_body_sha1,http_request_url,http_failure,failure_class,failure_str_full,failure_str,failure_str_raw,explorer_url
0,20240401T003750Z_webconnectivity_MM_136255_n1_4F0EJvo3L8aJzlPF,https://twitter.com/,20240401003850.725383_MM_webconnectivity_59c5e512edc11b74,MM,AS136255,Telecom International Myanmar Company Limited,MM,wifi,2024-04-01 00:38:49,twitter.com,web_connectivity,104.244.42.193,443.0,13414.0,Twitter Inc.,74.125.190.131,SG,15169,Google LLC,US,system,None,104.244.42.193,0.0,generic_timeout_error,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,None,None,None,None,tcp,tcp.generic_timeout_error,generic_timeout_error,generic_timeout_error,https://explorer.ooni.org/m/20240401003850.725383_MM_webconnectivity_59c5e512edc11b74
1,20240401T003750Z_webconnectivity_MM_136255_n1_4F0EJvo3L8aJzlPF,https://twitter.com/,20240401003850.725383_MM_webconnectivity_59c5e512edc11b74,MM,AS136255,Telecom International Myanmar Company Limited,MM,wifi,2024-04-01 00:38:49,twitter.com,web_connectivity,104.244.42.1,443.0,13414.0,Twitter Inc.,74.125.190.131,SG,15169,Google LLC,US,system,None,104.244.42.1,0.0,generic_timeout_error,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,None,None,None,None,tcp,tcp.generic_timeout_error,generic_timeout_error,generic_timeout_error,https://explorer.ooni.org/m/20240401003850.725383_MM_webconnectivity_59c5e512edc11b74
2,20240401T003750Z_webconnectivity_MM_136255_n1_4F0EJvo3L8aJzlPF,https://twitter.com/,20240401003850.725383_MM_webconnectivity_59c5e512edc11b74,MM,AS136255,Telecom International Myanmar Company Limited,MM,wifi,2024-04-01 00:38:49,twitter.com,web_connectivity,104.244.42.65,443.0,13414.0,Twitter Inc.,74.125.190.131,SG,15169,Google LLC,US,system,None,104.244.42.65,0.0,generic_timeout_error,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,None,None,None,None,tcp,tcp.generic_timeout_error,generic_timeout_error,generic_timeout_error,https://explorer.ooni.org/m/20240401003850.725383_MM_webconnectivity_59c5e512edc11b74
3,20240401T003750Z_webconnectivity_MM_136255_n1_4F0EJvo3L8aJzlPF,https://twitter.com/,20240401003850.725383_MM_webconnectivity_59c5e512edc11b74,MM,AS136255,Telecom International Myanmar Company Limited,MM,wifi,2024-04-01 00:38:49,twitter.com,web_connectivity,104.244.42.129,443.0,13414.0,Twitter Inc.,74.125.190.131,SG,15169,Google LLC,US,system,None,104.244.42.129,0.0,generic_timeout_error,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,None,None,None,None,tcp,tcp.generic_timeout_error,generic_timeout_error,generic_timeout_error,https://explorer.ooni.org/m/20240401003850.725383_MM_webconnectivity_59c5e512edc11b74
4,20240401T003750Z_webconnectivity_MM_136255_n1_4F0EJvo3L8aJzlPF,https://twitter.com/,20240401003850.725383_MM_webconnectivity_59c5e512edc11b74,MM,AS136255,Telecom International Myanmar Company Limited,MM,wifi,2024-04-01 00:38:49,twitter.com,web_connectivity,None,NaN,NaN,None,74.125.190.131,SG,15169,Google LLC,US,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,None,None,https://twitter.com/,generic_timeout_error,https,https.generic_timeout_error,generic_timeout_error,generic_timeout_error,https://explorer.ooni.org/m/20

In [ ]:
#df_with_failure = pd.read_csv('2024-MM-df_with_failure.csv')

In [35]:
len(df_with_failure)

128871

In [36]:
list(df_with_failure)

['report_id',
 'input',
 'measurement_uid',
 'probe_cc',
 'probe_asn',
 'probe_as_org_name',
 'probe_as_cc',
 'network_type',
 'measurement_start_time',
 'hostname',
 'test_name',
 'ip',
 'port',
 'ip_asn',
 'ip_as_org_name',
 'resolver_ip',
 'resolver_cc',
 'resolver_asn',
 'resolver_as_org_name',
 'resolver_as_cc',
 'dns_engine',
 'dns_failure',
 'dns_answer',
 'tcp_success',
 'tcp_failure',
 'tls_handshake_time',
 'tls_handshake_read_count',
 'tls_handshake_write_count',
 'tls_handshake_read_bytes',
 'tls_handshake_write_bytes',
 'tls_handshake_last_operation',
 'tls_success',
 'tls_is_certificate_valid',
 'tls_end_entity_certificate_subject',
 'tls_end_entity_certificate_subject_common_name',
 'tls_end_entity_certificate_issuer',
 'tls_end_entity_certificate_issuer_common_name',
 'tls_end_entity_certificate_san_list',
 'tls_end_entity_certificate_not_valid_after',
 'tls_end_entity_certificate_not_valid_before',
 'tls_failure',
 'http_response_body_sha1',
 'http_request_url',
 'http

In [52]:
# see: https://learn.microsoft.com/en-us/windows/win32/winsock/windows-sockets-error-codes-2
unknown_failure_map = {
    ': server misbehaving': 'dns_server_misbehaving',
    ': read: connection refused': 'connection_refused',
    ': connect: network is unreachable': 'network_unreachable',
    'tls: first record does not look like a TLS handshake': 'tls_bad_first_record',
    'remote error: tls: handshake failure': 'tls_handshake_failure',
    'remote error: tls: illegal parameter': 'tls_illegal_parameter',
    'connectex: No connection could be made because the target machine actively refused it': 'connection_refused',
    'read: connection refused': 'connection_refused',
    'remote error: tls: access denied': 'tls_access_denied',
    'remote error: tls: internal error': 'tls_internal_error',
    'HTTP/1.x transport connection broken: malformed HTTP version': 'http_malformed_response',
    'net/http: timeout awaiting response headers': 'http_timeout',
    'read: operation timed out': 'timed_out',
    'connect: operation timed out': 'timed_out',
    ': No address associated with hostname': 'dns_nxdomain_error',
    ': connect: bad file descriptor': 'bad_file_descriptor',
    'stream error: stream ID': 'http_stream_error',
    
    # This looks more like a golang-bug: https://github.com/golang/go/issues/31259
    'readLoopPeekFailLocked: <nil>': 'http_golang_bug',

    ': connect: no route to host': 'host_unreachable',
    ': connect: cannot assign requested address': 'address_not_available',
    'getaddrinfow: The requested name is valid, but no data of the requested type was found.': 'dns_no_answer',
    'wsarecv: Se ha forzado la interrupción de una conexión existente por el host remoto.': 'connection_reset',
    'wsarecv: An existing connection was forcibly closed by the remote host.': 'connection_reset',
    'wsarecv: Connessione in corso interrotta forzatamente dall\'host remoto.': 'connection_reset',
    'wsarecv: Uma ligação existente foi forçada a fechar pelo anfitrião remoto': 'connection_reset',
    
    'getaddrinfow: Ceci est habituellement une erreur temporaire qui se produit durant la résolution du nom d’hôte et qui signifie que le serveur local n’a pas reçu de réponse d’un serveur faisant autorité': 'dns_temporary_failure',
    'getaddrinfow: Dies ist normalerweise ein zeitweiliger Fehler bei der Auflösung von Hostnamen. Grund ist, dass der lokale Server keine Rückmeldung vom autorisierenden Server erhalten hat.': 'dns_temporary_failure',
    'getaddrinfow: Este é geralmente um erro temporário durante a resolução de nomes de anfitrião e significa que o servidor local não recebeu uma resposta de um servidor autoritário': 'dns_temporary_failure',
    'getaddrinfow: Éste es normalmente un error temporal durante la resolución de nombres de host y significa que el servidor local no recibió una respuesta de un servidor autoritativo': 'dns_temporary_failure',
}
def map_unknown_failure(failure_str):
    if not failure_str.startswith("unknown_failure"):
        return failure_str
    for substring, clean_failure in unknown_failure_map.items():
        if substring in failure_str:
            return clean_failure
    return "unknown_failure"

def simplify_failure(failure_str):
    if failure_str in ['timed_out', 'generic_timeout_error', 'deferred_timeout_error']:
        return 'timeout'
    
    if failure_str in ['android_dns_cache_no_data', 'dns_nxdomain_error']:
        return 'nxdomain'
    
    if failure_str in ['connection_refused', 'connection_refused_error']:
        return 'connection_refused'
        
    return failure_str

ipv6_failures = ['address_not_available', 'address_family_not_supported', 'network_unreachable', 'host_unreachable']
def compute_analysis(row):
    failure_str = map_unknown_failure(row['failure_str_raw'])

    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in confirmed_ips:
        return 'dns.confirmed'
    
    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in likely_ips:
        return 'dns.likely.blocked'
    
    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in false_positive_ips:
        return 'ok'
        
    if row['tls_is_certificate_valid'] == True:
        return 'ok'

    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in confirmed_http_resp:
        return 'http.confirmed'
    
    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in likely_http_resp:
        return 'http.likely.blocked'
    
    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in false_positive_http_resp:
        return 'ok'
    
    if row['failure_class'] == 'ok':
        return 'ok'

    if row['dns_consistency'] == 'inconsistent':
        return 'dns.inconsistent'
    
    if row['ip_as_org_name'] == 'Bogon':
        return 'dns.bogon'
    
    #if row['dns_blocking_scope'] not in ('u', 'n'):
    #    return f"dns.{row['dns_blocking_scope']}"
    
    if row['tls_is_certificate_valid'] == False:
        return 'tls.bad_cert'
    
    simple_failure = simplify_failure(failure_str)
    if simple_failure in ipv6_failures and row['ip'] and ':' in row['ip']:
        return 'ipv6_error'

    if simple_failure.startswith("ssl_"):
        simple_failure = 'bad_cert'
    
    prefix = row['failure_class']
    if prefix == 'https' and simple_failure.startswith('dns_') or simple_failure == 'nxdomain':
        prefix = 'dns'

    return f'{prefix}.{simple_failure}'

In [39]:
df_ctrl_dns_obs_web = click_query("""
SELECT 
groupUniqArray(ip) as uniq_ip,
groupUniqArray(ip_asn) as uniq_asn,
groupUniqArray(ip_as_org_name) as uniq_as_org_name,
hostname
FROM obs_web
WHERE measurement_start_time > %(measurement_start_day)s
AND probe_cc NOT IN %(cc_list)s
AND tls_is_certificate_valid = 1 
GROUP BY hostname
""", **{
    "measurement_start_day": MEASUREMENT_START_DAY,
    "cc_list": ANALYSIS_COUNTRY_CODES,
})

In [40]:
%%time
df_ctrl_dns_obs_web_ctrl = click_query("""
SELECT 
groupUniqArray(ip) as uniq_ip,
groupUniqArray(ip_asn) as uniq_asn,
groupUniqArray(ip_as_org_name) as uniq_as_org_name,
hostname
FROM obs_web_ctrl
WHERE measurement_start_time > %(measurement_start_day)s
AND tls_success = 1 
GROUP BY hostname
""", **{
    "measurement_start_day": MEASUREMENT_START_DAY
})

CPU times: user 660 ms, sys: 74.6 ms, total: 735 ms
Wall time: 31.8 s


In [41]:
from collections import defaultdict
ctrl_map = defaultdict(dict)
for _, row in df_ctrl_dns_obs_web_ctrl.iterrows():
    ctrl_map[row['hostname']]['uniq_ip_ctrl'] = set(row['uniq_ip'])
    ctrl_map[row['hostname']]['uniq_asn_ctrl'] = set(row['uniq_asn'])
    ctrl_map[row['hostname']]['uniq_as_org_name_ctrl'] = set(row['uniq_as_org_name'])
    
for _, row in df_ctrl_dns_obs_web.iterrows():
    ctrl_map[row['hostname']]['uniq_ip_web'] = set(row['uniq_ip'])
    ctrl_map[row['hostname']]['uniq_asn_web'] = set(row['uniq_asn'])
    ctrl_map[row['hostname']]['uniq_as_org_name_web'] = set(row['uniq_as_org_name'])

In [42]:
df_ctrl_dns_obs_web.head()

,uniq_ip,uniq_asn,uniq_as_org_name,hostname
0,[2a06:98c1:3121::3],[13335],[Cloudflare Inc],desk.im
1,"[46.107.16.55, 46.107.16.54, 2001:4c48:16:6::1:20, 2001:4c48:16:6::2:20]",[5483],[Magyar Telekom plc.],videa.hu
2,[185.34.219.59],[198203],[RouteLabel V.O.F.],women.jo
3,"[172.67.179.43, 104.21.31.185]",[13335],[Cloudflare Inc],es.zenit.org
4,"[2606:50c0:8002::153, 185.199.110.153, 185.199.109.153, 2606:50c0:8001::153, 185.199.108.153, 185.199.111.153, 2606:50c0:8000::153, 2606:50c0:8003::153]",[54113],"[Fastly, Inc.]",sibrea.github.io


In [43]:
df_ctrl_dns_obs_web_ctrl.head()

,uniq_ip,uniq_asn,uniq_as_org_name,hostname
0,[91.107.202.180],[24940],[Hetzner Online GmbH],foi.am
1,[154.17.26.106],[906],[DMIT Cloud Services],x18r.com
2,"[104.21.88.99, 2606:4700:3030::ac43:b04d, 2606:4700:3035::6815:5863, 172.67.176.77]",[13335],[Cloudflare Inc],ahang.hu
3,"[2a00:c70:624::935c:f3ce, 147.92.243.206]","[21409, 38631]","[Ikoula Net SAS, LINE Corporation]",line.me
4,"[104.21.38.78, 2606:4700:3035::ac43:dc17, 2606:4700:3036::6815:264e, 172.67.220.23]",[13335],[Cloudflare Inc],taiwantimes.net


In [44]:
def compute_dns_consistency(row):
    if pd.isnull(row['ip']):
        return None
    ctrl = ctrl_map[row['hostname']]
    if len(ctrl):
        return 'unknown'
    if row['ip'] in ctrl.get('uniq_ip_ctrl', {}):        
        return 'consistent.ip_ctrl'
    if row['ip'] in ctrl.get('uniq_ip_web', {}):
        return 'consistent.ip_web'
    if row['ip_asn'] in ctrl.get('uniq_asn_ctrl', {}):
        return 'consistent.asn_ctrl'
    if row['ip_asn'] in ctrl.get('uniq_asn_web', {}):
        return 'consistent.asn_web'
    if row['ip_as_org_name'] in ctrl.get('uniq_as_org_name_ctrl', {}):
        return 'consistent.as_org_name_ctrl'
    if row['ip_as_org_name'] in ctrl.get('uniq_as_org_name_web', {}):
        return 'consistent.as_org_name_web'
    return 'inconsistent'

In [45]:
df_with_failure['dns_consistency'] = df_with_failure.progress_apply(compute_dns_consistency, axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 128871/128871 [00:01<00:00, 88964.33it/s]


In [46]:
df_with_failure['analysis'] = df_with_failure.progress_apply(compute_analysis, axis=1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 128871/128871 [00:03<00:00, 35416.43it/s]


In [47]:
df_with_failure.to_csv('2024-MM-df_with_failure.csv')

In [48]:
confirmed_blocking = df_with_failure[
    (df_with_failure['analysis'] == 'dns.confirmed') | 
    (df_with_failure['analysis'] == 'http.confirmed')]

msmt_counts_confirmed = confirmed_blocking.pivot_table(index=['input','probe_cc'], values=['measurement_uid', 'analysis', 'probe_asn'],
                                                aggfunc={'measurement_uid': 'nunique', 
                                                         'analysis': lambda x: ', '.join(set(x)),
                                                         'probe_asn': lambda x: ', '.join(set(x))}).reset_index().rename(columns = {
                                                                    'measurement_uid': 'blocking_count',
                                                                    'analysis' : 'blocking_method',
                                                                    'probe_asn' : 'probe_asn_blocking'})

In [49]:
likely_blocking = df_with_failure[
    (df_with_failure['analysis'] == 'dns.likely.blocked') |
    (df_with_failure['analysis'] == 'http.likely.blocked')]

msmt_counts_likely = likely_blocking.pivot_table(index=['input','probe_cc'], values=['measurement_uid', 'analysis', 'probe_asn'],
                                                aggfunc={'measurement_uid': 'nunique', 
                                                         'analysis': lambda x: ', '.join(set(x)),
                                                         'probe_asn': lambda x: ', '.join(set(x))}).reset_index().rename(columns = {
                                                                    'measurement_uid': 'blocking_count',
                                                                    'analysis' : 'blocking_method',
                                                                    'probe_asn' : 'probe_asn_blocking'})

In [50]:
msmt_counts_confirmed.to_csv('2024-myanmmar-confirmed-blocked-heuristics.csv')

In [51]:
msmt_counts_likely.to_csv('2024-myanmmar-likely-blocked-heuristics.csv')

In [102]:
df_with_failure[
    df_with_failure['failure_str_raw'].str.startswith('unknown_failure')
][['failure_str_raw', 'analysis']].drop_duplicates()

,failure_str_raw,analysis
2079250,unknown_failure: connection error: PROTOCOL_ERROR,https.unknown_failure
2936041,unknown_failure: http2: timeout awaiting response headers,https.unknown_failure
4486740,unknown_failure: write tcp [scrubbed]->[scrubbed]: write: broken pipe,tls.unknown_failure
4764350,unknown_failure: write tcp [scrubbed]->[scrubbed]: write: broken pipe,https.unknown_failure
4886103,unknown_failure: remote error: tls: protocol version not supported,https.unknown_failure
5589254,unknown_failure: tls: server selected unsupported protocol version 301,dns.inconsistent
8581295,unknown_failure: remote error: tls: error decoding message,tls.unknown_failure


In [103]:
df_with_failure[
    df_with_failure['ip_as_org_name'] == 'Bogon'
]

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_handshake_time,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_success,tls_is_certificate_valid,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_certificate_chain_length,tls_failure,http_request_url,http_failure,http_runtime,failure_class,failure_str_full,failure_str,failure_str_raw,dns_consistency,analysis
1547,20230712T092917Z_webconnectivity_VN_18403_n1_tyMXuHYsWsEfDXD5,https://tambao.info/amp/,20230712094113.888004_VN_webconnectivity_6d308ad541d14711,VN,18403,FPT Telecom Company,VN,unknown,2023-07-12 09:41:13,tambao.info,web_connectivity,127.0.0.1,443.0,64666.0,Bogon,210.245.24.101,VN,18403,FPT Telecom Company,VN,system,None,127.0.0.1,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.connection_refused,connection_refused,connection_refused,unknown,dns.bogon
1608,20230712T092917Z_webconnectivity_VN_18403_n1_tyMXuHYsWsEfDXD5,https://the88project.org/,20230712094147.204673_VN_webconnectivity_686f917155123bf0,VN,18403,FPT Telecom Company,VN,unknown,2023-07-12 09:41:47,the88project.org,web_connectivity,::1,443.0,64666.0,Bogon,210.245.24.101,VN,18403,FPT Telecom Company,VN,system,None,::1,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.connection_refused,connection_refused,connection_refused,unknown,dns.bogon
1620,20230712T092917Z_webconnectivity_VN_18403_n1_tyMXuHYsWsEfDXD5,https://thoibao.com/,20230712094159.885458_VN_webconnectivity_cb4e963b348de059,VN,18403,FPT Telecom Company,VN,unknown,2023-07-12 09:41:59,thoibao.com,web_connectivity,::1,443.0,64666.0,Bogon,210.245.24.101,VN,18403,FPT Telecom Company,VN,system,None,::1,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.connection_refused,connection_refused,connection_refused,unknown,dns.bogon
1632,20230712T092917Z_webconnectivity_VN_18403_n1_tyMXuHYsWsEfDXD5,https://tiengnoidanchu.wordpress.com/,20230712094231.778810_VN_webconnectivity_af87e22bc6b77a1c,VN,18403,FPT Telecom Company,VN,unknown,2023-07-12 09:42:31,tiengnoidanchu.wordpress.com,web_connectivity,127.0.0.1,443.0,64666.0,Bogon,210.245.24.101,VN,18403,FPT Telecom Company,VN,system,None,127.0.0.1,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.connection_refused,connection_refused,connection_refused,unknown,dns.bogon
1633,20230712T092917Z_webconnectivity_VN_18403_n1_tyMXuHYsWsEfDXD5,https://tiengnoidanchu.wordpress.com/,20230712094231.778810_VN_webconnectivity_af87e22bc6b77a1c,VN,18403,FPT Telecom Company,VN,unknown,2023-07-12 09:42:31,tiengnoidanchu.wordpress.com,web_connectivity,::1,443.0,64666.0,Bogon,210.245.24.101,VN,18403,FPT Telecom Company,VN,system,None,::1,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.connection_refused,connection_refused,connection_refused,unknown,dns.bogon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10066206,20231109T193747Z_webconnectivity_VN_7552_n1_Uko6l2SKhRlC90Gk,https://baotiengdan.com/,20231109193836.752635_VN_webconnectivity_ddb1f4b090678c86,VN,7552,Viettel Group,VN,wifi,

In [104]:
df_with_failure[
    df_with_failure['analysis'] == 'dns.confirmed'
]

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_handshake_time,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_success,tls_is_certificate_valid,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_certificate_chain_length,tls_failure,http_request_url,http_failure,http_runtime,failure_class,failure_str_full,failure_str,failure_str_raw,dns_consistency,analysis
7024,20240302T031115Z_webconnectivity_VN_7552_n1_Y6YFgIgmts0VsR1V,http://www.tqlcvn.org/,20240302031322.815226_VN_webconnectivity_a92b08d7e018fcf0,VN,7552,Viettel Group,VN,mobile,2024-03-02 03:13:23,www.tqlcvn.org,web_connectivity,125.235.4.59,80.0,7552.0,Viettel Group,27.71.194.194,VN,7552,Viettel Group,VN,system,None,125.235.4.59,0.0,generic_timeout_error,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.generic_timeout_error,generic_timeout_error,generic_timeout_error,inconsistent,dns.confirmed
8678,20240302T033226Z_webconnectivity_VN_7552_n1_Xet7gZfkWNWS5xju,https://scontent-frt3-2.cdninstagram.com/favicon.ico,20240302033631.268289_VN_webconnectivity_23790dba62193cec,VN,7552,Viettel Group,VN,wifi,2024-03-02 03:36:30,scontent-frt3-2.cdninstagram.com,web_connectivity,125.235.4.59,443.0,7552.0,Viettel Group,27.68.251.238,VN,7552,Viettel Group,VN,system,None,125.235.4.59,0.0,generic_timeout_error,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.generic_timeout_error,generic_timeout_error,generic_timeout_error,inconsistent,dns.confirmed
9213,20240302T033226Z_webconnectivity_VN_7552_n1_Xet7gZfkWNWS5xju,https://www.ms88vtv.com/,20240302033845.448078_VN_webconnectivity_b4dba8dbaf3748ce,VN,7552,Viettel Group,VN,wifi,2024-03-02 03:38:44,www.ms88vtv.com,web_connectivity,125.212.193.174,443.0,38731.0,Vietel - CHT Compamy Ltd,27.68.251.238,VN,7552,Viettel Group,VN,system,None,125.212.193.174,1.0,None,0.048580,6.0,2.0,12528.0,288.0,write_2,1,0.0,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,[],2049-03-08 06:12:46,2021-10-22 06:12:46,1.0,ssl_invalid_hostname,None,None,NaN,tls,tls.ssl_invalid_hostname,ssl_invalid_hostname,ssl_invalid_hostname,unknown,dns.confirmed
9888,20240302T033226Z_webconnectivity_VN_7552_n1_Xet7gZfkWNWS5xju,https://www.vuicungdafa.com/vn/,20240302034111.762361_VN_webconnectivity_3dd4c9c34d1b6218,VN,7552,Viettel Group,VN,wifi,2024-03-02 03:41:10,www.vuicungdafa.com,web_connectivity,125.212.193.174,443.0,38731.0,Vietel - CHT Compamy Ltd,27.68.251.238,VN,7552,Viettel Group,VN,system,None,125.212.193.174,1.0,None,0.047289,4.0,2.0,12528.0,292.0,write_2,1,0.0,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,[],2049-03-08 06:12:46,2021-10-22 06:12:46,1.0,ssl_invalid_hostname,None,None,NaN,tls,tls.ssl_invalid_hostname,ssl_invalid_hostname,ssl_invalid_hostname,inconsistent,dns.confirmed
58002,20240302T062617Z_webconnectivity_VN_7552_n1_1O3DZ0IQ9xVrd57J,https://vnhrdmay11.org/,20240302062817.607006_VN_webconnectivity_02c6299da4f1f3f0,VN,7552,Viettel Group,VN,wifi,2024-03-02 06:28:15,vnhrdmay11.org,web_connectivity,125.235.4.59,443.0,7552.0,Viettel Group,27.71.195.66,VN,75

In [105]:
len(df_with_failure['failure_str_full'].unique())

54

In [106]:
len(df_with_failure['analysis'].unique())

45

In [107]:
df_with_failure[
    (df_with_failure['probe_cc'] == 'VN')
    & (df_with_failure['analysis'] != 'ipv6_error')
]

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_handshake_time,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_success,tls_is_certificate_valid,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_certificate_chain_length,tls_failure,http_request_url,http_failure,http_runtime,failure_class,failure_str_full,failure_str,failure_str_raw,dns_consistency,analysis
0,20240109T235539Z_webconnectivity_VN_7552_n1_aQEJI8CYCwIlhVtW,http://www.gayscape.com/,20240110000005.868806_VN_webconnectivity_803068196f56d4df,VN,7552,Viettel Group,VN,wifi,2024-01-10 00:00:02,www.gayscape.com,web_connectivity,204.8.234.161,80.0,395412.0,VS Media Inc.,172.253.4.2,HK,15169,Google LLC,US,system,None,204.8.234.161,1.0,None,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,ok,ok,,,unknown,ok
1,20240109T235539Z_webconnectivity_VN_7552_n1_aQEJI8CYCwIlhVtW,http://www.gayscape.com/,20240110000005.868806_VN_webconnectivity_803068196f56d4df,VN,7552,Viettel Group,VN,wifi,2024-01-10 00:00:02,www.gayscape.com,web_connectivity,None,NaN,NaN,None,172.253.4.2,HK,15169,Google LLC,US,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,https://www.gayscape.com/,None,NaN,ok,ok,,,None,ok
2,20240109T235539Z_webconnectivity_VN_7552_n1_aQEJI8CYCwIlhVtW,http://www.gayscape.com/,20240110000005.868806_VN_webconnectivity_803068196f56d4df,VN,7552,Viettel Group,VN,wifi,2024-01-10 00:00:02,www.gayscape.com,web_connectivity,None,NaN,NaN,None,172.253.4.2,HK,15169,Google LLC,US,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,http://www.gayscape.com/,None,NaN,ok,ok,,,None,ok
3,20240109T235539Z_webconnectivity_VN_7552_n1_aQEJI8CYCwIlhVtW,https://www.gaystarnews.com/,20240110000006.989721_VN_webconnectivity_8f2e27c327ff67a5,VN,7552,Viettel Group,VN,wifi,2024-01-10 00:00:06,www.gaystarnews.com,web_connectivity,104.21.234.125,443.0,13335.0,Cloudflare Inc,172.253.4.2,HK,15169,Google LLC,US,system,None,104.21.234.125,1.0,None,0.085150,2.0,4.0,5552.0,397.0,write_4,1,1.0,CN=gaystarnews.com,gaystarnews.com,"CN=GTS CA 1P5,O=Google Trust Services LLC,C=US",GTS CA 1P5,"[gaystarnews.com, *.gaystarnews.com]",2024-04-07 08:26:03,2024-01-08 08:26:04,3.0,None,None,None,NaN,ok,ok,,,unknown,ok
4,20240109T235539Z_webconnectivity_VN_7552_n1_aQEJI8CYCwIlhVtW,https://www.gaystarnews.com/,20240110000006.989721_VN_webconnectivity_8f2e27c327ff67a5,VN,7552,Viettel Group,VN,wifi,2024-01-10 00:00:06,www.gaystarnews.com,web_connectivity,104.21.234.124,443.0,13335.0,Cloudflare Inc,172.253.4.2,HK,15169,Google LLC,US,system,None,104.21.234.124,1.0,None,0.093481,2.0,4.0,5552.0,397.0,write_4,1,1.0,CN=gaystarnews.com,gaystarnews.com,"CN=GTS CA 1P5,O=Google Trust Services LLC,C=US",GTS CA 1P5,"[gaystarnews.com, *.gaystarnews.com]",2024-04-07 08:26:03,2024-01-08 08:26:04,3.0,None,None,None,NaN,ok,ok,,,unknown,ok
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10069620,20231109T203520Z_webconnectivity_VN_133606_n1_QLEbDWGC6LrxTnox,https://www.vice.com/,20231109210122.902658_VN_webconnectivity_397a21ea447095d8,VN,133606,Viettel Timor Leste,TL,wifi,2023-11-09 21:01:12,www.vice.com,web_connectivity,151.101.66.133,443.0,54113.0,"Fastly, Inc.",172.253.215.197,BE,15169,Google LLC,US,syst

In [108]:
df_with_failure[
     (df_with_failure['analysis'] != 'ok')
]['analysis'].unique()

array(['ipv6_error', 'https.eof_error', 'https.timeout', 'tls.timeout',
       'dns.inconsistent', 'https.bad_cert', 'dns.nxdomain',
       'tls.eof_error', 'https.connection_refused', 'tls.bad_cert',
       'tcp.timeout', 'https.host_unreachable', 'dns.timeout',
       'dns.bogon', 'tls.connection_reset', 'https.connection_reset',
       'tcp.host_unreachable', 'tcp.connection_refused', 'dns.confirmed',
       'dns.dns_no_answer', 'https.tls_illegal_parameter',
       'dns.dns_server_misbehaving', 'https.network_unreachable',
       'tcp.connection_reset', 'https.http_stream_error',
       'tls.internal_error', 'https.internal_error',
       'https.connection_aborted', 'tls.invalid_record',
       'https.invalid_record', 'tcp.connection_aborted',
       'tcp.permission_denied', 'https.http_malformed_response',
       'dns.dns_temporary_failure', 'tls.connection_aborted',
       'https.unknown_failure', 'tls.tls_handshake_failure',
       'https.tls_handshake_failure', 'tcp.network_unr

In [53]:
%%time
target_domains = [
    "www.facebook.com"
]
df_special = click_query("""
WITH multiIf(
    dns_failure IS NOT NULL, tuple('dns', dns_failure),
    tcp_failure IS NOT NULL, tuple('tcp', tcp_failure),
    tls_failure IS NOT NULL, tuple('tls', tls_failure),
    http_failure IS NOT NULL, tuple('https', http_failure),
    tuple('ok', '')
) as failure
SELECT 
report_id,
input,
measurement_uid,
probe_cc,
probe_asn,
probe_as_org_name,
probe_as_cc,
network_type,
measurement_start_time,
hostname,
test_name,
ip,
port,
ip_asn,
ip_as_org_name,
resolver_ip,
resolver_cc,
resolver_asn,
resolver_as_org_name,
resolver_as_cc,
dns_engine,
dns_failure,
dns_answer,
tcp_success,
tcp_failure,
tls_handshake_time,
tls_handshake_read_count,
tls_handshake_write_count,
tls_handshake_read_bytes,
tls_handshake_write_bytes,
tls_handshake_last_operation,
tls_cipher_suite IS NOT NULL as tls_success,
tls_is_certificate_valid,
tls_end_entity_certificate_subject,
tls_end_entity_certificate_subject_common_name,
tls_end_entity_certificate_issuer,
tls_end_entity_certificate_issuer_common_name,
tls_end_entity_certificate_san_list,
tls_end_entity_certificate_not_valid_after,
tls_end_entity_certificate_not_valid_before,
tls_certificate_chain_length,
tls_failure,
http_response_body_sha1,
http_request_url,
http_failure,
failure.1 as failure_class,
IF(failure_class = 'ok', 'ok', concat(failure_class, '.', failure_str)) as failure_str_full,
IF(startsWith(failure.2, 'unknown_failure'), 'unknown_failure', failure.2) as failure_str,
failure.2 as failure_str_raw
FROM obs_web
WHERE probe_cc = 'MM' 
AND measurement_start_time > '2023-07-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'web_connectivity'
AND hostname IN %(target_domains)s
""", target_domains=target_domains)

CPU times: user 15.2 s, sys: 1.06 s, total: 16.3 s
Wall time: 1min 14s


In [54]:
len(df_special)

21632

In [55]:
df_special['dns_consistency'] = df_special.progress_apply(compute_dns_consistency, axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21632/21632 [00:00<00:00, 132282.22it/s]


In [56]:
df_special['analysis'] = df_special.progress_apply(compute_analysis, axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21632/21632 [00:00<00:00, 23209.42it/s]


In [57]:
df_special["probe_asn"] = "AS" + df_special["probe_asn"].apply(str)

In [58]:
method_of_blocking = df_special[df_special['analysis'] != 'ok'].pivot_table(index=['analysis'], values=['measurement_uid'], columns=['probe_asn','probe_as_org_name'],
                                                aggfunc={'measurement_uid': len}).fillna(0).reset_index()

method_of_blocking

analysis           measurement_uid  \
probe_asn                                                      AS132100   
probe_as_org_name                             Myanmar Speed Net Co.,Ltd   
0                                   dns.bogon                       0.0   
1                               dns.confirmed                       0.0   
2                  dns.dns_server_misbehaving                       0.0   
3                   dns.dns_temporary_failure                       0.0   
4                                dns.nxdomain                       0.0   
5                                 dns.timeout                       0.0   
6                    https.connection_aborted                       0.0   
7                    https.connection_refused                       3.0   
8                      https.connection_reset                       0.0   
9                             https.eof_error                       0.0   
10                  https.network_unreachable                       0.0   
11                              https.timeout                      10.0   
12                                 ipv6_error                      12.0   
13                     tcp.connection_aborted                       0.0   
14                     tcp.connection_refused                       4.0   
15                      tcp.permission_denied                       0.0   
16                                tcp.timeout                      12.0   
17                       tls.connection_reset                       0.0   
18                              tls.eof_error                       0.0   
19                                tls.timeout                       0.0   

                                                                 \
probe_asn                                              AS132148   
probe_as_org_name HORIZON TELECOM INTERNATIONAL COMPANY LIMITED   
0                                                           0.0   
1                                                           0.0   
2                                                           0.0   
3                                                           0.0   
4                                                           0.0   
5                                                           0.0   
6                                                           0.0   
7                                                           0.0   
8                                                           0.0   
9                                                           0.0   
10                                                          0.0   
11                                                          8.0   
12                                                         10.0   
13                                                          0.0   
14                                                          0.0   
15                                                          0.0   
16                                                          9.0   
17                                                          0.0   
18                                                          0.0   
19                                                          0.0   

                                                                       \
probe_asn                        AS132167                    AS133384   
probe_as_org_name Ooredoo Myanmar Limited Global Technology Co., Ltd.   
0                                     0.0                         0.0   
1                                     0.0                         0.0   
2                                     0.0                         0.0   
3                                     0.0                         0.0   
4                                    52.0                         2.0   
5                                     4.0                         2.0   
6                                     0.0                         0.0   
7                                     0

In [15]:
%%time
df_signal = click_query("""
SELECT *
FROM obs_web
WHERE probe_cc = 'MM' 
AND measurement_start_time > '2024-04-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'signal'
""")

CPU times: user 2.48 s, sys: 179 ms, total: 2.66 s
Wall time: 1min 20s


In [64]:
df_signal

,measurement_uid,input,report_id,measurement_start_time,software_name,software_version,test_name,test_version,probe_asn,probe_cc,probe_as_org_name,probe_as_cc,probe_as_name,network_type,platform,origin,engine_name,engine_version,architecture,resolver_ip,resolver_asn,resolver_cc,resolver_as_org_name,resolver_as_cc,resolver_is_scrubbed,resolver_asn_probe,resolver_as_org_name_probe,bucket_date,observation_id,created_at,post_processed_at,target_id,hostname,transaction_id,ip,port,ip_asn,ip_as_org_name,ip_as_cc,ip_cc,ip_is_bogon,dns_query_type,dns_failure,dns_engine,dns_engine_resolver_address,dns_answer_type,dns_answer,dns_answer_asn,dns_answer_as_org_name,dns_t,tcp_failure,tcp_success,tcp_t,tls_failure,tls_server_name,tls_version,tls_cipher_suite,tls_is_certificate_valid,tls_end_entity_certificate_fingerprint,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_certificate_chain_length,tls_certificate_chain_fingerprints,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_handshake_time,tls_t,http_request_url,http_network,http_alpn,http_failure,http_request_body_length,http_request_method,http_runtime,http_response_body_length,http_response_body_is_truncated,http_response_body_sha1,http_response_status_code,http_response_header_location,http_response_header_server,http_request_redirect_from,http_request_body_is_truncated,http_t,probe_analysis,pp_http_response_fingerprints,pp_http_fingerprint_country_consistent,pp_http_response_matches_blockpage,pp_http_response_matches_false_positive,pp_http_response_body_title,pp_http_response_body_meta_title,pp_dns_fingerprint_id,pp_dns_fingerprint_country_consistent
0,20240110010110.131971_MM_signal_dc8851915b8a27d2,None,20240110T010108Z_signal_MM_134739_n1_LR5G3vPB4TooCfIl,2024-01-10 01:01:09,ooniprobe-desktop-unattended,3.19.2,signal,0.2.5,134739,MM,AGB Communication Co.Ltd,MM,20151020,unknown,macos,unknown,ooniprobe-engine,3.19.2,amd64,172.71.209.73,13335,HK,Cloudflare Inc,US,0,13335,Cloudflare Inc,2024-01-10,20240110010110.131971_MM_signal_dc8851915b8a27d2_0,2024-01-11 03:15:17,None,None,sfu.voip.signal.org,None,34.36.20.21,443.0,396982.0,Google LLC,US,US,0.0,A,None,system,,A,34.36.20.21,396982.0,Google LLC,0.118038,None,1.0,0.165971,None,sfu.voip.signal.org,TLSv1.3,TLS_AES_128_GCM_SHA256,1.0,9ee18b05a48dfa26ccaeb747121cfcd709e9b99ed890a46c8ace2574cb51ce97,,,"CN=Signal Messenger,O=Signal Messenger\, LLC,L=Mountain View,ST=California,C=US",Signal Messenger,[sfu.voip.signal.org],2024-10-01 01:40:37,2023-08-31 19:51:52,2.0,"[9ee18b05a48dfa26ccaeb747121cfcd709e9b99ed890a46c8ace2574cb51ce97, ddb0f92bb95c8d6fd202ea6e8cc5ccd182b544f8cd696f47d580659ddc9df65a]",6.0,12.0,3613.0,1109.0,write_12,0.040518,0.230383,None,None,None,None,NaN,None,None,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,[],None,0,0,None,None,None,None
1,20240110010110.131971_MM_signal_dc8851915b8a27d2,None,20240110T010108Z_signal_MM_134739_n1_LR5G3vPB4TooCfIl,2024-01-10 01:01:09,ooniprobe-desktop-unattended,3.19.2,signal,0.2.5,134739,MM,AGB Communication Co.Ltd,MM,20151020,unknown,macos,unknown,ooniprobe-engine,3.19.2,amd64,172.71.209.73,13335,HK,Cloudflare Inc,US,0,13335,Cloudflare Inc,2024-01-10,20240110010110.131971_MM_signal_dc8851915b8a27d2_1,2024-01-11 03:15:17,None,None,sfu.voip.signal.org,None,2600:1901:0:57c3::,NaN,396982.0,Google LLC,US,US,0.0,AAAA,None,system,,AAAA,2600:1901:0:57c3::,396982.0,Google LLC,0.118038,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,[],NaT,NaT,NaN,[],NaN,NaN,NaN,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,NaN,NaN,None,NaN,None,None,None,NaN,NaN,None,[],None,0,0,None,None,None,None
2,20240110010110.131971_MM_signal_dc8851915b8a27d2,None,20240110T010108Z_signal_MM

In [18]:
%%time
df_fb = click_query("""
SELECT *
FROM obs_web
WHERE probe_cc = 'MM' 
AND measurement_start_time > '2023-07-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'facebook_messenger'

""")

CPU times: user 647 ms, sys: 56.2 ms, total: 703 ms
Wall time: 54 s


In [20]:
df_fb

""
